In [1]:
import numpy as np
from keras.layers import *
from keras.models import *

Using TensorFlow backend.


In [2]:
#rnn -> (batchsize, timestep, inputdim)
x=np.array([[[1,2,3], [4,5,6]],
         [[7,8,9], [10,11,12]],
         [[13,14,15], [16,17,18]]])
x
#rnn->(batchsize(3),timesteps(2),inputdim(3))
              

array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]])

In [3]:
#simple rnn: 2개의 (timesteps, inputdim)
rnn=SimpleRNN(50)(Input(shape=(10,30)))
#                        (timestep, inputdim)
#return_sequences = False로 디폴트 설정
#=> 출력 값이 셀의 개수와 동일, 셀 하나당
#하나의 스칼라값이 리턴
print(rnn.shape)

W0925 14:34:44.848307   668 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0925 14:34:44.863266   668 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0925 14:34:44.866274   668 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



(?, 50)


In [4]:
#simple rnn: 2개의 (timesteps, inputdim)
rnn=SimpleRNN(50,return_sequences = True )(Input(shape=(10,30)))
#                        (timestep, inputdim)
#return_sequences = True로 설정
#timestep * num of cell 개가 출력
print(rnn.shape)


(?, ?, 50)


In [5]:
rnn=SimpleRNN(50, return_sequences=False)(Input(shape=(10,30)))
print(rnn)

Tensor("simple_rnn_3/TensorArrayReadV3:0", shape=(?, 50), dtype=float32)


In [6]:
rnn=SimpleRNN(50, return_sequences=True)(Input(shape=(10,30)))
print(rnn)

Tensor("simple_rnn_4/transpose_1:0", shape=(?, ?, 50), dtype=float32)


In [7]:
from keras import *
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
from konlpy.tag import Okt

In [8]:
chatbot_data=pd.read_csv("ChatbotData.csv")
question, answer=list(chatbot_data['Q']),list(chatbot_data['A'])

In [9]:
#한 문장에서 단어 시퀀스의 최대 개수
max_sequences=30

#임베딩 벡터 차원
embedding_dim=100
# [0 0 1 0 .... 0 0 0] => [1.5 2.7 3.8 1.7...]
#  <----450여개---->      <------100개------>

#LSTM 히든레이어 차원
lstm_hidden_dim=128

#정규표현식 필터
RE_FILTER=re.compile("[.,!?\"':;~()]")

In [13]:
chatbot_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [10]:
PAD="<PADDING>" #패딩
STA="<START>" #시작
END="<END>" #끝
OOV="<OOV>" #없는 단어

#태그 문자에 대한 인덱스 정의
PAD_INDEX=0
STA_INDEX=1
END_INDEX=2
OOV_INDEX=3

#데이터 타입
ENCODER_INPUT=0
DECODER_INPUT=1
DECODER_TARGET=2

In [14]:
#                      잘   가   eos (DECODER_TARGET)
# 안   녕   하 세 요
# 인코더         ->       디코더
# hello(ENCODER_INPUT) STA  잘   가 (DECODER_INPUT)

In [15]:
len(question)

11823

In [16]:
question=question[:100]
answer=answer[:100]

In [17]:
for i in range(10):
    print("Qustion:" + question[i])
    print("answer:" + answer[i])
    print()

Qustion:12시 땡!
answer:하루가 또 가네요.

Qustion:1지망 학교 떨어졌어
answer:위로해 드립니다.

Qustion:3박4일 놀러가고 싶다
answer:여행은 언제나 좋죠.

Qustion:3박4일 정도 놀러가고 싶다
answer:여행은 언제나 좋죠.

Qustion:PPL 심하네
answer:눈살이 찌푸려지죠.

Qustion:SD카드 망가졌어
answer:다시 새로 사는 게 마음 편해요.

Qustion:SD카드 안돼
answer:다시 새로 사는 게 마음 편해요.

Qustion:SNS 맞팔 왜 안하지ㅠㅠ
answer:잘 모르고 있을 수도 있어요.

Qustion:SNS 시간낭비인 거 아는데 매일 하는 중
answer:시간을 정하고 해보세요.

Qustion:SNS 시간낭비인데 자꾸 보게됨
answer:시간을 정하고 해보세요.



In [18]:
#1. 단어사전 생성
#문장 -> 토큰(글자, 단어, 문장) 분리 -> KoNLPy(한국어) -> 형태소분석기(한나눔, 코모란, Okt 등)
#KoNLPy(자바로 작성되어있음) 사용하기 위해 
#1)java설치(>=1.7)->환경변수 이름:JAVA_HOME, 값:c:\program files\jdk....
#2)jpype1 설치(파이썬에서 자바를 호출할 수 있도록 하는 라이브러리)  #아나콘다 프롬프트 -> pip install jpype파일저장경로
#3)KoNLPy 설치 => 아나콘다 프롬프트 -> pip install KoNLPy



In [19]:
#형태소 분석 함수
def pos_tag(sentences):
    tagger=Okt()
    sentences_pos=[]
    for sentence in sentences:
        sentence=re.sub(RE_FILTER,"",sentence)
        sentence=" ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
    #print(sentences_pos)
    return sentences_pos

In [20]:
question=pos_tag(question)
question
answer=pos_tag(answer)
answer

C:\Users\user\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


['하루 가 또 가네요',
 '위로 해 드립니다',
 '여행 은 언제나 좋죠',
 '여행 은 언제나 좋죠',
 '눈살 이 찌푸려지죠',
 '다시 새로 사는 게 마음 편해요',
 '다시 새로 사는 게 마음 편해요',
 '잘 모르고 있을 수도 있어요',
 '시간 을 정 하고 해보세요',
 '시간 을 정 하고 해보세요',
 '자랑 하는 자리 니까 요',
 '그 사람 도 그럴 거 예요',
 '그 사람 도 그럴 거 예요',
 '혼자 를 즐기세요',
 '돈 은 다시 들어올 거 예요',
 '땀 을 식혀주세요',
 '어서 잊고 새 출발 하세요',
 '빨리 집 에 돌아가서 끄고 나오세요',
 '빨리 집 에 돌아가서 끄고 나오세요',
 '다음 달 에는 더 절약 해봐요',
 '따뜻하게 사세요',
 '다음 달 에는 더 절약 해봐요',
 '가장 확실한 시간 은 오늘이 에요 어제 와 내일 을 놓고 고민 하느라 시간 을 낭비하지 마세요',
 '온 가족 이 모두 마음 에 드는 곳 으로 가보세요',
 '온 가족 이 모두 마음 에 드는 곳 으로 가보세요',
 '온 가족 이 모두 마음 에 드는 곳 으로 가보세요',
 '저 를 만들어 준 사람 을 부모님 저 랑 이야기 해 주는 사람 을 친구 로 생각 하고 있어요',
 '저 를 만들어 준 사람 을 부모님 저 랑 이야기 해 주는 사람 을 친구 로 생각 하고 있어요',
 '더 가까워질 기회 가 되겠네요',
 '저 도 요',
 '다 들 바빠서 이야기 할 시간 이 부족했나 봐요',
 '다 들 바빠서 이야기 할 시간 이 부족했나 봐요',
 '온 가족 이 모두 마음 에 드는 곳 으로 가보세요',
 '좋은 생각 이에요',
 '더 가까워질 기회 가 되겠네요',
 '저 를 만들어 준 사람 을 부모님 저 랑 이야기 해 주는 사람 을 친구 로 생각 하고 있어요',
 '좋은 생각 이에요',
 '정말 후회 할 습관 이에요',
 '무모한 결정 을 내 리지 마세요',
 '선생님 이나 기관 에 연락 해보세요',
 '떨리는 감정 은 그 자체 로 소중해요',
 '득템 했길 바라요

In [23]:
for i in range(10):
    print("Qustion:" + question[i])
    print("answer:" + answer[i])
    print()

Qustion:12시 땡
answer:하루 가 또 가네요

Qustion:1 지망 학교 떨어졌어
answer:위로 해 드립니다

Qustion:3 박 4일 놀러 가고 싶다
answer:여행 은 언제나 좋죠

Qustion:3 박 4일 정도 놀러 가고 싶다
answer:여행 은 언제나 좋죠

Qustion:PPL 심하네
answer:눈살 이 찌푸려지죠

Qustion:SD 카드 망가졌어
answer:다시 새로 사는 게 마음 편해요

Qustion:SD 카드 안 돼
answer:다시 새로 사는 게 마음 편해요

Qustion:SNS 맞팔 왜 안 하지 ㅠㅠ
answer:잘 모르고 있을 수도 있어요

Qustion:SNS 시간 낭비 인 거 아는데 매일 하는 중
answer:시간 을 정 하고 해보세요

Qustion:SNS 시간 낭비 인데 자꾸 보게 됨
answer:시간 을 정 하고 해보세요



In [24]:
sentences=[]
sentences.extend(question)

In [25]:
sentences.extend(answer)
sentences

['12시 땡',
 '1 지망 학교 떨어졌어',
 '3 박 4일 놀러 가고 싶다',
 '3 박 4일 정도 놀러 가고 싶다',
 'PPL 심하네',
 'SD 카드 망가졌어',
 'SD 카드 안 돼',
 'SNS 맞팔 왜 안 하지 ㅠㅠ',
 'SNS 시간 낭비 인 거 아는데 매일 하는 중',
 'SNS 시간 낭비 인데 자꾸 보게 됨',
 'SNS 보면 나 만 빼고 다 행복 해보여',
 '가끔 궁금해',
 '가끔 뭐 하는지 궁금해',
 '가끔 은 혼자 인게 좋다',
 '가난한 자의 설움',
 '가만 있어도 땀 난다',
 '가상 화폐 쫄딱 망함',
 '가스 불 켜고 나갔어',
 '가스 불 켜놓고 나온거 같아',
 '가스 비 너무 많이 나왔다',
 '가스 비 비싼데 감기 걸리겠어',
 '가스 비 장난 아님',
 '가장 확실한 건 뭘 까',
 '가족 여행 가기 로 했어',
 '가족 여행 고고',
 '가족 여행 어디 로 가지',
 '가족 있어',
 '가족 관계 알려 줘',
 '가족 끼리 여행 간다',
 '가족 들 보고 싶어',
 '가족 들 이랑 서먹해',
 '가족 들 이랑 서먹해졌어',
 '가족 들 이랑 어디 가지',
 '가족 들 이랑 여행 갈거야',
 '가족 여행 가야 지',
 '가족 이 누구 야',
 '가족 이랑 여행 가려고',
 '가족 한테 스트레스 풀었어',
 '가출 할까',
 '가출 해도 갈 데 가 없어',
 '간만 에 떨리니까 좋더라',
 '간만 에 쇼핑 중',
 '간만 에 휴식 중',
 '간식 뭐 먹을까',
 '간식 추천',
 '간장 치킨 시켜야지',
 '간접흡연 싫어',
 '갈까 말까 고민 돼',
 '갈까 말까',
 '감 말랭이 먹고 싶다',
 '감 말랭이 먹어야지',
 '감기 같 애',
 '감기 걸린 것 같아',
 '감기 기운 이 있어',
 '감기 들 거 같 애',
 '감기 가 오려나',
 '감기 약 이 없어',
 '감기 인거 같 애',
 '감미로운 목소리 좋아',
 '감정 이 쓰레기통 처럼 엉망 진창 이야',
 '감정 컨트롤 을 못 하겠어',
 '감정 컨

In [26]:
words=[]
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

        #길이가 0인 단어는 제거
words=[word for word in words if len(word)>0]

#중복된 단어 제거
words=list(set(words))
len(words)
#가장 앞부분에 태그 삽입
words[:0]=[PAD, STA, END, OOV]

In [27]:
words
#단어 <-> 인덱스로 구성된 딕셔너리
word_to_index={word:index for index, word in enumerate(words)}
index_to_word={index:word for index, word in enumerate(words)}
# word_to_index={'<PADDING>':0, ...}   
# index_to_word={0:'<PADDING>', ...}

In [28]:
list(word_to_index.items())[:20]
list(index_to_word.items())[:20]

[(0, '<PADDING>'),
 (1, '<START>'),
 (2, '<END>'),
 (3, '<OOV>'),
 (4, '다시'),
 (5, '질질'),
 (6, '못'),
 (7, '있는'),
 (8, '매일'),
 (9, '집어서'),
 (10, '남겨야'),
 (11, '생각'),
 (12, '키우고'),
 (13, '있을'),
 (14, '왜'),
 (15, '씨방'),
 (16, '반'),
 (17, '바라요'),
 (18, '서로'),
 (19, '입어볼까')]

In [29]:
#seq2seq는 학습시 아래와 같은 3개의 데이터가 필요
#인코더 입력:12시 땡
#디코더 입력:start 점심먹으러 가요
#디코더 출력:점심먹으러 가요 end
#훈련 모델과 예측 모델이 따로 구성되어야 함.

In [30]:
def convert_text_to_index(sentences,vocabulary,type):
    sentences_index=[]
    for sentence in sentences:
        sentence_index=[]        
        #디코더 입력-> 맨 앞에 START 추가
        if type==DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])
        for word in sentence.split():
            #사전에 있는 단어면 인덱스 추가
            if vocabulary.get(word) is not None:
                sentence_index.extend([vocabulary[word]])
            else:
                sentence_index.extend([vocabulary[OOV]])
        
        if type==DECODER_TARGET:
            #DECODER_TARGET -> 맨 뒤에 END 추가
            if len(sentence_index) >= max_sequences:
                sentence_index=sentence_index[:max_sequences-1]+[vocabulary[END]]
            else:
                sentence_index+=[vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index= sentence_index[:max_sequences]
        sentence_index+=(max_sequences-len(sentence_index))*[vocabulary[PAD]]
        sentences_index.append(sentence_index)
    return np.asarray(sentences_index)                                                     

In [31]:
x_encoder=convert_text_to_index(question, word_to_index, ENCODER_INPUT) #문장 -> 인덱스
x_encoder[0] #전체 문장에 대한 인덱스 변환 결과가 출력
#12시 땡 -> [5, 80, 0, 0, ... , 0]
#한 문장에서 단어 시퀀스의 최대 개수:max_sequences=30

array([384, 165,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [32]:
#디코더 입력 인덱스 변환
x_decoder=convert_text_to_index(answer, word_to_index, DECODER_INPUT) #문장 -> 인덱스
x_decoder[0] #start 점심먹으러 가요

array([  1, 175, 358,  41, 435,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [33]:
#디코더 target(목표) 인덱스 변환
y_decoder=convert_text_to_index(answer, word_to_index, DECODER_TARGET) #문장 -> 인덱스
y_decoder[0] #점심먹으러 가요 end

array([175, 358,  41, 435,   2,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [34]:
#2. 전처리(문장 -> 인덱스 변환)

In [35]:
#인코더입력, 디코더입력은 임베딩계층에 들어가는 인덱스 배열
#디코더출력은 원핫인코딩 형식으로 출력
len(y_decoder)  #(100,30,455)
#100:답변문장의 개수, 30:시퀀스 길이, 455:전체단어 개수(중복제외
one_hot_data=np.zeros((len(y_decoder), max_sequences, len(words)))
#디코더 target을 원핫인코딩으로 변환
#학습할때는 입력은 인덱스, 출력은 원핫인코딩 형식
one_hot_data.shape# (100(문장,i),30(단어개수,j),455(index))  #455 => 원핫인코딩
for i, sequence in enumerate(y_decoder):
    #print(i)
     for j, index in enumerate(sequence):
            #print(index)
            one_hot_data[i,j,index]=1
y_decoder=one_hot_data
y_decoder[0]  #(100,30,455)




array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [36]:
#훈련 모델 정의

#훈련 모델 인코더 정의:입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs=Input(shape=(None,))
#임베딩 계층
encoder_outputs=Embedding(len(words), embedding_dim)(encoder_inputs)
encoder_outputs, state_h, state_c=LSTM(lstm_hidden_dim,
    dropout=0.1,
    return_state=True)(encoder_outputs)

encoder_states=[state_h, state_c]


#훈련 모델 디코더 정의:목표 문장의 시퀀스를 입력으로 받음
decoder_inputs=Input(shape=(None,))
decoder_embedding=Embedding(len(words), embedding_dim)
decoder_outputs=decoder_embedding(decoder_inputs)

#return_sequences=True, 모든 time step출력값 리턴
decoder_lstm=LSTM(lstm_hidden_dim,
    dropout=0.1,
    return_state=True,
    return_sequences=True)

decoder_outputs, _, _=decoder_lstm(decoder_outputs, initial_state=encoder_states)

#단어의 개수만큼 노드의 개수를 설정. 원핫 형식으로 각 단어 인덱스
decoder_dense=Dense(len(words), activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)

#훈련 모델 정의

model=models.Model([encoder_inputs,decoder_inputs],decoder_outputs)

#학습 방법 설정
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])


W0925 14:36:22.508929   668 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0925 14:36:22.513914   668 deprecation.py:506] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0925 14:36:22.881929   668 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0925 14:36:22.897890   668 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The n

In [37]:
#예측 모델 정의
#훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
encoder_model=models.Model(encoder_inputs,encoder_states)

#예측 모델 디코더 정의
#예측할때는 훈련할때와는 달리 타임 스텝을 한 단계씩 수행
#매번 이전 디코더의 상태를 입력으로 받아서 새로 설정

decoder_state_input_h=Input(shape=(lstm_hidden_dim,))
decoder_state_input_c=Input(shape=(lstm_hidden_dim,))
decoder_state_inputs=[decoder_state_input_h, decoder_state_input_c]

#임베딩 계층
decoder_outputs=decoder_embedding(decoder_inputs)

#lstm 계층
decoder_outputs, state_h, state_c=decoder_lstm(decoder_outputs, initial_state=decoder_state_inputs)

#히든 상태와 셀 상태를 하나로 묶음
decoder_states=[state_h, state_c]

#dense를 사용하여 원핫형식으로 인덱스 표현
decoder_outputs=decoder_dense(decoder_outputs)

#예측 모델 디코더 설정
decoder_model=models.Model([decoder_inputs]+decoder_state_inputs,
            [decoder_outputs]+decoder_states)


In [38]:
#훈련 및 테스트

#인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary):
    sentence=""
    #print(index)   
    for index in indexs:
        if index==END_INDEX: #종료 인덱스 -> 중단
            break;
        if vocabulary.get(index) is not None:
            sentence+=vocabulary[index]
        else:
            sentence.extend([vocabulary[OOV_INDEX]])
        sentence += " "
    #문장으로 변환
    return sentence


In [39]:
#model.fit()
#x_encoder[2] => y+deocder[2]

for epoch in range(20):
    print("total epoch:", epoch+1)
    
    history=model.fit([x_encoder,x_decoder],y_decoder,
             epochs=100,
             batch_size=64,
             verbose=0)
    print('accuracy:', history.history['acc'][-1])
    print('cost:', history.history['loss'][-1])    
    
    #3박 4일 놀러 가고 싶다 ->여행 은 언제나 좋죠
    input_encoder=x_encoder[2].reshape(1,x_encoder[2].shape[0])
    input_decoder=x_decoder[2].reshape(1,x_decoder[2].shape[0])
    results=model.predict([input_encoder,input_decoder])
    
    indexs=np.argmax(results[0], 1)
    sentence=convert_index_to_text(indexs, index_to_word)
    print(sentence)
    print()  
    


W0925 14:36:23.398578   668 deprecation.py:323] From C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


total epoch: 1
accuracy: 0.838999981880188
cost: 0.6936711359024048
저 도 도 요 

total epoch: 2
accuracy: 0.9399999809265137
cost: 0.29758179545402524
저 은 언제나 좋죠 

total epoch: 3
accuracy: 0.9693333458900452
cost: 0.11949900686740875
가세 은 언제나 좋죠 

total epoch: 4
accuracy: 0.9773333287239074
cost: 0.07730905205011368
저 은 언제나 좋죠 

total epoch: 5
accuracy: 0.9803333282470703
cost: 0.05940972775220871
저 은 언제나 좋죠 

total epoch: 6
accuracy: 0.9860000085830688
cost: 0.04527069091796875
저 은 언제나 좋죠 

total epoch: 7
accuracy: 0.9870000052452087
cost: 0.042196324914693835
여행 은 언제나 좋죠 

total epoch: 8
accuracy: 0.9913333344459534
cost: 0.029427510350942612
여행 은 언제나 좋죠 

total epoch: 9
accuracy: 0.9976666522026062
cost: 0.011044026985764503
여행 은 언제나 좋죠 

total epoch: 10
accuracy: 0.9966666555404663
cost: 0.011066766306757927
여행 은 언제나 좋죠 

total epoch: 11
accuracy: 1.0
cost: 0.0016608151374384761
여행 은 언제나 좋죠 

total epoch: 12
accuracy: 0.9986666488647461
cost: 0.004573833281174302
여행 은 언제나 좋죠 

total e

In [40]:
# input_encoder=x_encoder[2].reshape(1,x_encoder[2].shape[0])
# input_decoder=x_decoder[2].reshape(1,x_decoder[2].shape[0])
# results=model.predict([input_encode,input_decoder])

#x_decoder[2].shape[0]

In [41]:
LSTM(3, input_dim=1, input_length=4)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """Entry point for launching an IPython kernel.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(3, input_shape=(4, 1))`
  """Entry point for launching an IPython kernel.


In [42]:
Dense(3, input_dim=1)

In [43]:
#예측을 위한 입력 생성
def make_predict_input(sentence):
    sentences=[]
    sentences.append(sentence)
    sentences=pos_tag(sentences)
    input_seq=convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    return input_seq

In [44]:
# 텍스트 생성
def generate_text(input_seq):
    #입력을 인코더에 넣어서 마지막 상태가 구해짐
    states=encoder_model.predict(input_seq)
    
    #목표 시퀀스 초기화
    target_seq=np.zeros((1,1))
    
    #목표 시퀀스의 첫번째 입력: <START>
    target_seq[0,0]=STA_INDEX
    
    indexs=[]
    while 1:
        decoder_outputs, state_h, state_c=decoder_model.predict([target_seq]+states)
        #결과가 원핫인코딩 형식인데 인덱스로 변환함
        index=np.argmax(decoder_outputs[0,0,:])
        indexs.append(index)
        #종료조건 검사
        if index==END_INDEX or len(indexs)>=max_sequences:
            break
        #목표 시퀀스를 바로 이전의 출력으로 설정
        
        
        #목표 시퀀스 초기화
        target_seq=np.zeros((1,1))
        #목표 시퀀스의 첫번째 입력: <START>
        target_seq[0,0]=index
        
        #디코더의 이전상태를 다음 디코더 예측에 사용
        states=[state_h, state_c]
    
    #인덱스 집합이 문장으로 변환
    sentence=convert_index_to_text(indexs, index_to_word)
    
    return sentence

In [45]:
input_seq=make_predict_input("집에 있을꺼야 3박 4일")
input_seq
#[[300, 150, 170, ..., 0, 0, 0]]

array([[ 72, 399,   3, 437, 379, 346,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [46]:
sentence=generate_text(input_seq)
print(sentence)
#여행은 언제나 좋죠

가장 확실한 시간 은 오늘이 에요 어제 와 내일 을 놓고 고민 하느라 시간 을 낭비하지 마세요 


In [47]:
input_seq=make_predict_input("어디 가고 싶은 데 있어?")
input_seq
#[[300, 150, 170, ..., 0, 0, 0]]

array([[ 85, 288,   3, 433, 101,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [48]:
sentence=generate_text(input_seq)
print(sentence)
#여행은 언제나 좋죠

자신 을 더 사랑 해주세요 


In [74]:
input_seq=make_predict_input("")
input_seq
#[[300, 150, 170, ..., 0, 0, 0]]

array([[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]])

In [75]:
sentence=generate_text(input_seq)
print(sentence)
#여행은 언제나 좋죠

곧 방학 이 예요 


In [69]:
chatbot_data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.,0
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.,0
7,SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.,0
8,SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.,0
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.,0
